In [1]:
import httplib2
import os
from time import sleep

from apiclient import discovery
from oauth2client import client
from oauth2client import tools
from oauth2client.file import Storage

import logging
logging.basicConfig(filename='example.log',level=logging.DEBUG)

files_list = []
folders_list = []

In [2]:

home_dir = os.path.expanduser('~')
credential_dir = os.path.join(home_dir, '.credentials')
if not os.path.exists(credential_dir):
    os.makedirs(credential_dir)
credential_path = os.path.join(credential_dir, 'gDriveConnect.json')

print('credential_path:', credential_path)

credential_path: /global/home/users/mmanning/.credentials/gDriveConnect.json


In [3]:
SCOPES = 'https://www.googleapis.com/auth/drive.metadata.readonly'
CLIENT_SECRET_FILE = 'client_secret.json'
APPLICATION_NAME = 'gDriveConnect'

In [4]:
def get_credentials():
    flow = client.flow_from_clientsecrets(CLIENT_SECRET_FILE, SCOPES)
    flow.user_agent = APPLICATION_NAME
    if flags:
        credentials = tools.run_flow(flow, store, flags)
    else: # Needed only for compatibility with Python 2.6
        credentials = tools.run(flow, store)
    print('Storing credentials to ' + credential_path)
    return credentials

In [5]:
def process_subfolders(folder_name, folder_id):
    result = []

    param = {}
    template = "'{}' in parents and mimeType='application/vnd.google-apps.folder'"
    query = template.format(str(folder_id))
    param['pageSize'] = 1000
    param['q'] = query
    
    page_token = None
    while True:
        try:

            if page_token:
                param['pageToken'] = page_token
            
            sleep(0.25) # Time in seconds.
            folders = service.files().list(**param).execute()

            #print('folders in this folder:', len(folders['files']) )
            result.extend(folders['files'])
            page_token = folders.get('nextPageToken')
            if not page_token:
                break
        #except Exception as inst :
        except    errors.HttpError as error :
          print('An error occurred:', error.args)
          break

    print('process_subfolders result size:', len(result) )
    #print('process_subfolders result:', result )

    for folder in result:
        folders_list.append( (folder_name, folder_id, folder['name'], folder['id'])) 
        
        if ( folder_id == folder['id'] ):
            print('ERROR !!!!!!!!!!!!!!!!!!!!!!!!!!!!', folder['name'],)
        
        
        add_pdf_files(folder['name'], folder['id'])
        
        process_subfolders(folder['name'], folder['id'])


In [6]:
def add_pdf_files(folder_name, folder_id):
    param = {}
    template = "'{}' in parents and mimeType='application/pdf'"
    query = template.format(str(folder_id))
    #print('query', query)
    
    #param['q'] = "'0B9Ibqa26YXiRNmk2WS0tNmhjejg' in parents and mimeType='application/vnd.google-apps.folder'"
    param['q'] = query
    param['pageSize'] = 1000
    
    result = []
    page_token = None
    while True:
        #print('add_pdf_files: still true')
        try:

            if page_token:
                param['pageToken'] = page_token
                
            sleep(0.25) # Time in seconds.
            files = service.files().list(**param).execute()
            print('files:', len(files['files']) )

            result.extend(files['files'])
            page_token = files.get('nextPageToken')
            #print('files  nextPageToken:', files.get('nextPageToken') )
            if not page_token:
                break
        #except Exception as inst :
        except    errors.HttpError as error :
          print('An error occurred:', error.args)
          break
    
    for item in result:
        files_list.append( (folder_name, folder_id, item['name'], item['id'])) 
        #print('{0} ({1})'.format(item['name'], item['id']))

    print('files_list:', len(files_list) )


In [7]:
import argparse
import logging

parser = argparse.ArgumentParser(parents=[tools.argparser])
parser.add_argument('-f', help=argparse.SUPPRESS)
#parser.add_argument('--auth_host_name')
#parser.add_argument('--noauth_local_webserver', action='store_true', default=True, help='Do not run a local web server.')
#flags = argparse.ArgumentParser(parents=[parent]).parse_args()
flags = parser.parse_known_args()[0]
flags.noauth_local_webserver = True

#flags = parser.parse_args()
print('flags:', flags)
logging.basicConfig(level='WARNING')

flags: Namespace(auth_host_name='localhost', auth_host_port=[8080, 8090], f='/global/home/users/mmanning/.local/share/jupyter/runtime/kernel-e3647962-faf4-4430-8fdd-40a986cac85c.json', logging_level='ERROR', noauth_local_webserver=True)


In [8]:
store = Storage(credential_path)
print('credential_path:', credential_path)
credentials = get_credentials()
print('credentials:', credentials)
#http = credentials.authorize(httplib2.Http())
#service = discovery.build('drive', 'v3', http=http)

credential_path: /global/home/users/mmanning/.credentials/gDriveConnect.json

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&client_id=231667401550-d66jcj0dsafrvt6mu05grksevti1mcqh.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.metadata.readonly&access_type=offline&response_type=code

Enter verification code: 4/3FdZrIufNJc_MBirSHfL_qAzn0_y2Z0qSNeH3_DqqBk
Authentication successful.
Storing credentials to /global/home/users/mmanning/.credentials/gDriveConnect.json
credentials: <oauth2client.client.OAuth2Credentials object at 0x2b48d0225b70>


In [9]:
print('credential_path:', credential_path)
http = credentials.authorize(httplib2.Http())
service = discovery.build('drive', 'v3', http=http)

credential_path: /global/home/users/mmanning/.credentials/gDriveConnect.json


In [ ]:
# https://developers.google.com/drive/v2/reference/files/list

# ane.pdf.share (0B9Ibqa26YXiReTZaVm9OWmNxSlk)
# To be sorted... (0B9Ibqa26YXiRNmk2WS0tNmhjejg)
# Delete Uploads (to free up space) (0ByFFNduW4doJODEzOThmNWQtNjI2Ni00MTEyLTg0NDUtMDA4MmM1Mzg1NDM0)

# first get the pdfs in these top level folders
add_pdf_files('ane.pdf.share', '0B9Ibqa26YXiReTZaVm9OWmNxSlk') 
add_pdf_files('To be sorted...', '0B9Ibqa26YXiRNmk2WS0tNmhjejg') 

# then recurse thru the subfolders
process_subfolders('ane.pdf.share', '0B9Ibqa26YXiReTZaVm9OWmNxSlk') 
process_subfolders('To be sorted...', '0B9Ibqa26YXiRNmk2WS0tNmhjejg') 

print('files_list_final:', len(files_list) )
print('folder_list_final:', len(folders_list) )

with open('allfiles.txt', 'w', encoding="utf-8") as outfile:
    for file in files_list:
        outfile.write( ''.join([ file[0],';', file[1],';', file[2],';', file[3], '\n' ]) ) 
outfile.close()

with open('allfolders.txt', 'w', encoding="utf-8") as outfile2:
    for folder in folders_list:
        outfile2.write( ''.join([ folder[0],';', folder[1],';', folder[2],';', folder[3], '\n' ]) ) 
outfile2.close()


files: 0
files_list: 2646
files: 441
files_list: 3087
process_subfolders result size: 4
files: 0
files_list: 3087
process_subfolders result size: 2
files: 1
files_list: 3088
process_subfolders result size: 26
files: 1000
files: 218
files_list: 4306
process_subfolders result size: 0
files: 652
files_list: 4958
process_subfolders result size: 0
files: 90
files_list: 5048
process_subfolders result size: 0
files: 409
files_list: 5457
process_subfolders result size: 0
files: 771
files_list: 6228
process_subfolders result size: 0
files: 602
files_list: 6830
process_subfolders result size: 0
files: 300
files_list: 7130
process_subfolders result size: 0
files: 753
files_list: 7883
process_subfolders result size: 0
files: 928
files_list: 8811
process_subfolders result size: 0
files: 685
files_list: 9496
process_subfolders result size: 0
files: 851
files_list: 10347
process_subfolders result size: 0
files: 948
files_list: 11295
process_subfolders result size: 0
files: 1000
files: 304
files_list: